In [7]:
from stsci.tools import capable
capable.OF_GRAPHICS = False

from pyraf import iraf
from pyraf.iraf import gemini, gemtools, gmos, onedspec
import fileSelect as fs
import copy

In [8]:
pwd

u'/home/mmarcano/Documents/GeminiReduceTTU/Object6'

First specified the directory where all the data was downloaded and extracted. Then defined the sqlite3 database. This contains the information of all the files. This obsLog.sqlite3 file was created by the python script obslog.py in the raw data directory. The script will create a database with the fields and metadata

In [9]:
datadirall = '../raw/'
databasename = datadirall+'obsLog.sqlite3'
dbFile= datadirall+'obsLog.sqlite3'

 From the work_directory:
 Create the query dictionary of essential parameter=value pairs.

In [10]:
qd = {'Full':{'use_me':1,
       'Instrument':'GMOS-S','CcdBin':'2 2','RoI':'Full',
       'Disperser':'R400+_G5325','CentWave':656.0,'AperMask':'0.5arcsec',
       'Object':'Obj-6',
       'DateObs':'2018-09-20:2018-09-30'}
      }


 In the tutorial there are two regions but here only one and it is full. Just changed the regions list to only Full. It should work. Bias only depend on:
 instrument Roi and CcdBin according to fileSelect file
 
     use_me=1 AND ObsType='BIAS' AND ObsClass LIKE '%Cal'
    AND Instrument=:Instrument AND RoI=:RoI AND CcdBin=:CcdBin

In [11]:
gemtools.gemextn.unlearn()    # Disarm a bug in gbias
gmos.gbias.unlearn()
biasFlags = {
    'logfile':'biasLog.txt','rawpath':'../raw/','fl_vardq':'yes','verbose':'no'
}
regions = ['Full']
for r in regions:
    # The following SQL generates the list of full-frame files to process.
    SQL = fs.createQuery('bias', qd[r])
    biasFiles = fs.fileListQuery(dbFile, SQL, qd[r])

    # The str.join() funciton is needed to transform a python list into a
    # comma-separated string of file names that IRAF can understand.
    if len(biasFiles) > 1:
        gmos.gbias(','.join(str(x) for x in biasFiles), 'MCbias'+r,
                   **biasFlags)

ERROR - GBIAS: Output file MCbiasFull.fits already exists.


![](oldbias.png)

# Flat Field

In [12]:
# Set the task parameters.
qd['Full'].update({'DateObs':'*'})
gmos.gireduce.unlearn()
gmos.gsflat.unlearn()
# The response fitting should be done interactively.
flatFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_dark':'no',
    'fl_fixpix':'no','fl_oversize':'no','fl_vardq':'yes','fl_fulldq':'yes',
    'rawpath':'../raw/','fl_inter':'no','fl_detec':'yes',
    'function':'spline3','order':'13,11,28',
    'logfile':'gsflatLog.txt','verbose':'no'
    }
for r in regions:
    qr = qd[r]
    flatFiles = fs.fileListQuery(dbFile, fs.createQuery('gcalFlat', qr), qr)
    if len(flatFiles) > 0:
        gmos.gsflat (','.join(str(x) for x in flatFiles), 'MCflat'+r,
                bias='MCbias'+r, **flatFlags)


GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    

# Basic Reduction

In [13]:
# Set task parameters.
gmos.gsreduce.unlearn()
sciFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_gscrrej':'no',
    'fl_dark':'no','fl_flat':'yes','fl_gmosaic':'yes','fl_fixpix':'no',
    'fl_gsappwave':'yes','fl_oversize':'no',
    'fl_vardq':'yes','fl_fulldq':'yes','rawpath':'../raw/',
    'fl_inter':'no','logfile':'gsreduceLog.txt','verbose':'no'
}
arcFlags = copy.deepcopy(sciFlags)
arcFlags.update({'fl_flat':'no','fl_vardq':'no','fl_fulldq':'no'})


# Arc exposures
for r in regions:
    qr = qd[r]
    arcFiles = fs.fileListQuery(dbFile, fs.createQuery('arc', qr), qr)
    if len(arcFiles) > 0:
        gmos.gsreduce (','.join(str(x) for x in arcFiles), bias='MCbias'+r,
                  **arcFlags)


# Science exposures
r = 'Full'
sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qd[r]), qd[r])
if len(sciFiles) > 0:
    gmos.gsreduce (','.join(str(x) for x in sciFiles), bias='MCbias'+r,
              flatim='MCflat'+r, **sciFlags)

GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged


# Try Cosmioc-ray

No sure if use science gS or science gsS or the combine one for object 5?

# Wavelength Calibration (Check this later)

**Image rectification and wavelength linearization depend upon the wavelength calibration, using the arc lamp exposures taken immediately before each sequence of science and standard star exposures** (see Wavelength Calibration). In this case, the default medium-resolution line list will work well. The fit to the dispersion relation should be performed interactively, but for expediency we will use a previously determined functional fit. 


I had tpo manually put the name of the two arc Files. I am not sure how may to use. 


- [ ] On that day there is only one arc


Science ones are:

- S20180925S0072.fits on 2018-09-25 at 01:41:16.4
- S20180925S0071.fits on 2018-09-25 at 01:28:12.4

The closet arc is on 2018-09-27 called :S20180928S0014.fits

In [14]:
# Set task parameters
gmos.gswavelength.unlearn()
waveFlags = {
    'coordlist':'gmos$data/CuAr_GMOS.dat','fwidth':6,'nsum':50,
    'function':'chebyshev','order':5,
    'fl_inter':'no','logfile':'gswaveLog.txt','verbose':'no'
    }
# Must select specific wavecals to match science exposures.
prefix = 'gs'
for arc in ['S20180928S0014']:
     gmos.gswavelength (prefix+arc, **waveFlags)

# Advanced Processing


In the example tutorial:

>The targets in this program were observed in 3 slit orientations, and a few exposures were obtained at each position. This provides an opportunity to combine the sequential exposures at each position to remove cosmic rays, rather than rejecting CRs on single frames using the gsreduce.fl_gscrrej+ flag or running the gemcrspec task. The combined exposures for each target are then wavelength calibrated, and sky subtracted. First set the processing parameters.

Here we have two exposures of the same thing. Supposed it is the same. 

In [18]:
# Set task parameters.
gemtools.gemcombine.unlearn()
sciCombFlags = {
    'combine':'average','reject':'ccdclip',
    'fl_vardq':'yes','fl_dqprop':'yes',
    'logfile':'gemcombineLog.txt.txt','verbose':'no'
}
stdCombFlags = copy.deepcopy(sciCombFlags)
stdCombFlags.update({'fl_vardq':'no','fl_dqprop':'no'})
gmos.gstransform.unlearn()
transFlags = {
    'fl_vardq':'yes','interptype':'linear','fl_flux':'yes',
    'logfile':'gstransLog.txt'
}
# The sky regions should be selected with care, using e.g. prows/pcols:
#   pcols ("tAM2306b.fits[SCI]", 1100, 2040, wy1=40, wy2=320)
gmos.gsskysub.unlearn()
skyFlags = {
    'fl_oversize':'no','fl_vardq':'yes','logfile':'gsskysubLog.txt'
}

# Science Targets

In [19]:
sciTargets = {
    'Obj-6':{'arc':'gsS20180928S0014','sky':'100:700'},
}
for targ,p in sciTargets.iteritems():
    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ[-1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    gemtools.gemcombine (','.join(prefix+str(x) for x in sciFiles),
                         sciOut, **sciCombFlags)
    gmos.gstransform (sciOut, wavtraname=p['arc'], **transFlags)
    gmos.gsskysub ('t'+sciOut, long_sample=p['sky'], **skyFlags)

WARNING - GEMCOMBINE: five or less images to combine.
-------------------------------------------------------------------------------
GSTRANSFORM -- Mon Nov  5 12:53:09 CST 2018
 

inimages   = Obj6
outimages  = 
outprefix  = t
fl_stran   = no
fl_wavtran = yes
wavtraname = gsS20180928S0014
database   = database
fl_vardq   = yes
interptype = linear
lambda1    = INDEF
lambda2    = INDEF
dx         = INDEF
nx         = INDEF
lambdalog  = no
ylog       = no
fl_flux    = yes
gratingdb  = gmos$data/GMOSgratings.dat
filterdb   = gmos$data/GMOSfilters.dat
key_dispaxis = DISPAXIS
dispaxis   = 1
sci_ext    = SCI
var_ext = VAR
dq_ext  = DQ

Transforming Obj6.fits[SCI,1]
MDF row: 1

NOAO/IRAF V2.16 mmarcano@tux Mon 12:53:10 05-Nov-2018
  Transform Obj6.fits[SCI,1] to tmpsci8816_1878.
  Conserve flux per pixel.
  User coordinate transformations:
    gsS20180928S0014_001
  Interpolation is linear.
  Using edge extension for out of bounds pixel values.
  Output coordinate parameters are:
    x1 =    


pcols tObj5.fits[SCI] 1500 1700 wy1=40 

prows tAM2306b.fits[SCI] 520 720 wcs='world'

![Pcols](pcolstobj5.png)

![Pcolszo](pcolzoom.png)

Sky http://www.gemini.edu/sciops/telescopes-and-sites/observing-condition-constraints/optical-sky-background

# To Do. 


Play with average of cloumns. TO show well a sky and an source

## Open as Multi Frame

Get slit coordinates and area and create a region

In [23]:
onedspec.nsum=2
onedspec.sarith('stObj6.fits[SCI]', 'copy', '', 'estObj6.ms',
                  apertures='1100-1200x2')

# Try cosmic Removal

# try with Obj6

In [44]:
sciTargets = {
    'Obj-6':{'arc':'gsS20180928S0014','sky':'100:700'},
}

cosmicFlags = {
    'niter':'4','fl_vardq':'yes','logfile':'gemcrspecLog.txt','sigfrac':'0.32'
}

gemfixFlags = {
    'method':'fixpix','logfile':'gemfixLog.txt'
}


for targ,p in sciTargets.iteritems():
    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ[-1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    gemtools.gemcombine (','.join(prefix+str(x) for x in sciFiles),
                         sciOut, **sciCombFlags)
    gmos.gstransform (sciOut, wavtraname=p['arc'], **transFlags)
    
    #Cosmic ray sbefre sky?
    gemtools.gemcrspec('t'+sciOut,'cosmict'+sciOut,**cosmicFlags)
    gemtools.gemfix('scosmict'+sciOut,'sfixcosmict'+sciOut,**gemfixFlags)


    gmos.gsskysub ('cosmict'+sciOut, long_sample=p['sky'], **skyFlags)



ERROR - GEMCOMBINE: Output file Obj6 already exists.
-------------------------------------------------------------------------------
GSTRANSFORM -- Tue Nov  6 12:15:39 CST 2018
 
ERROR - GSTRANSFORM: Output image tObj6 exists
ERROR - GSTRANSFORM: 1 errors found with input parameters. Exiting.

GSTRANSFORM exit status: error
-------------------------------------------------------------------------------

----------------------------------------------------------------------------
GSSKYSUB -- Tue Nov  6 12:15:39 CST 2018

input        = tObj6
output       = 
outpref      = s
fl_vardq     = yes
fl_oversize  = no
sci_ext      = SCI
var_ext      = VAR
dq_ext       = DQ
long_sample  = 100:700
mos_sample   = 0.9
mosobjsize   = 1.0
naverage     = 1
function     = chebyshev
order        = 1
low_reject   = 2.5
high_reject  = 2.5
niterate     = 2
grow         = 0.0
fl_inter     = no

ERROR - GSSKYSUB: Output image stObj6 exist
ERROR - GSSKYSUB: 1 error(s) found. Exiting.

GSSKYSUB done. Exit stat

In [45]:
extractobj='Obj6'
spec='sfixcosmict'+extractobj+'[SCI]'

In [46]:
onedspec.nsum=2
onedspec.sarith(spec, 'copy', '', 'e'+spec+'.ms',
                  apertures='1100-1200x2')